In [81]:
import os
from typing import Callable, List, Optional, Tuple

import attr
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as scipy_img
import scipy.interpolate as sci
from cv2 import VideoWriter, VideoWriter_fourcc

In [85]:
MARGIN = 2.0
MAX_RADIUS = 20.0
MAX_ATTEMPTS = 200
GROW_STEP = 3.0


@attr.dataclass
class Circle:
    x: float
    y: float
    r: float
    growing: bool

    def grow(self):
        if not self.growing:
            return
        if self.r >= MAX_RADIUS:
            self.growing = False
            return
        self.r += GROW_STEP

    def collides(self, others: List["Circle"]) -> Optional["Circle"]:
        # Check if this circle collides with any of the existing circles.
        # Since we only care about the comparison, we compare the distance squared
        # to avoid unnecessary square rooting (since this implementation is already
        # roughly O(N^2))
        for other in others:
            if other is self:
                continue
            r = self.r + other.r + MARGIN
            r_squared = r * r
            xdelta = abs(other.x - self.x)
            ydelta = abs(other.y - self.y)
            dist_squared = xdelta * xdelta + ydelta * ydelta
            if dist_squared <= r_squared:
                return other

        return None


@attr.dataclass
class Board:
    circles: List[Circle]
    width: float
    height: float

    @classmethod
    def initial(cls, width: float, height: float) -> "Board":
        return cls(circles=[], width=float(width), height=float(height))

    def new_circle(self) -> bool:
        for _attempt in range(MAX_ATTEMPTS):
            x = np.random.random() * self.width
            y = np.random.random() * self.height
            c = Circle(int(x), int(y), 1.0, True)
            if c.collides(self.circles):
                continue
            self.circles.append(c)
            return True
        return False

    def draw_circles(
        self,
        frame: np.ndarray,
        thickness: int,
        color_func: Callable[[Circle], Tuple[int, int, int]],
        radius_factor: float = 1.0,
    ) -> None:
        for circle in self.circles:
            cv2.circle(
                frame,
                (circle.x, circle.y),
                int(np.round(circle.r * radius_factor)),
                color=color_func(circle),
                thickness=thickness,
                lineType=cv2.LINE_AA,
            )

    def grow(self) -> None:
        for circle in self.circles:
            collision = circle.collides(self.circles)
            if collision:
                # print(f'circle {circle} collides with {collision}; stopping growth')
                circle.growing = False
            circle.grow()

In [90]:
width = 640
height = 480
FPS = 15
seconds = 10  # 1.0
frames = int(FPS * seconds)

# Could use "test.mp4" and the "H264" fourcc codec below.
FILENAME = "test.webm"
if os.path.exists(FILENAME):
    os.remove(FILENAME)

fourcc = VideoWriter_fourcc(*"VP80")
video = VideoWriter(FILENAME, fourcc, float(FPS), (width, height))

# Create the initial "board" which will contain packed circles.
board = Board.initial(width, height)

mod0 = 0.0

for frame_num in range(frames):
    frame = np.ones((height, width, 3), dtype=np.uint8) * 255

    # Attempt to pack in a new circle.
    board.new_circle()

    # Draw blurry circles, with some variation in size using the modulating variable.
    board.draw_circles(
        frame,
        thickness=-1,
        color_func=lambda c: (50, max(255, int(c.r * 10) + 120), 120),
        radius_factor=np.sin(mod0),
    )
    kernel = int(MAX_RADIUS * 4)
    kernel += 1 if kernel % 2 == 0 else 0
    frame = cv2.GaussianBlur(frame, (kernel, kernel), 0.0)

    # Draw circle outlines in thick black.
    board.draw_circles(frame, thickness=2, color_func=lambda _: (0, 0, 0))
    board.grow()
    video.write(frame)
    mod0 += np.pi / 180

video.release()

In [ ]:
from IPython.display import Video

Video(FILENAME, embed=True)